In [1]:
import subprocess
import json

In [2]:
def regex_equiv(gold, predicted):
    if gold == predicted:
        return True
    try:
        out = subprocess.check_output(['java', '-jar', 'regex_dfa_equals.jar', '{}'.format(gold), '{}'.format(predicted)])
#          print("out: {}".format(out))
        #print(out.decode('utf-8'))
        if '\n1' in out.decode('utf-8'):
            return True
        else:
            return False
    except Exception as e:
        return False


In [3]:

names = ["Phi_Raw_Output","Phi_Refined_Output"]
for name in names:
    print("Starting: ", name)
    filename = f'../Generation/{name}.json'
    outfilename = f'./{name}_DFA_Result.json'

    with open(filename, 'r') as f:
        data = json.load(f)
    print(len(data))

    dfa_result = []
    for item in data:
        print("Starting item: ", item['id'])
        id = item['id']
        gold_expression = item['expression']
        # print ("Expression: ", gold_expression)
        for i in range(len(item['phi_output'])):
            predicted_expression = item['phi_output'][i]['text']
            item['completion'] = predicted_expression
            passed = regex_equiv(gold_expression, predicted_expression)
            
            dfa_result.append({
                'id': id,
                'completion': predicted_expression,
                'passed': passed
            })  
        print("Ending item: ", item['id'])

    with open(outfilename, 'w') as f:
        json.dump(dfa_result, f, indent=4)

Starting:  Phi_Raw_Output
762
Starting item:  1
Ending item:  1
Starting item:  2
Ending item:  2
Starting item:  3
Ending item:  3
Starting item:  4
Ending item:  4
Starting item:  5
Ending item:  5
Starting item:  6
Ending item:  6
Starting item:  7
Ending item:  7
Starting item:  8
Ending item:  8
Starting item:  9
Ending item:  9
Starting item:  10
Ending item:  10
Starting item:  11
Ending item:  11
Starting item:  12
Ending item:  12
Starting item:  13
Ending item:  13
Starting item:  15
Ending item:  15
Starting item:  16
Ending item:  16
Starting item:  18
Ending item:  18
Starting item:  20
Ending item:  20
Starting item:  25
Ending item:  25
Starting item:  26
Ending item:  26
Starting item:  27
Ending item:  27
Starting item:  34
Ending item:  34
Starting item:  36
Ending item:  36
Starting item:  41
Ending item:  41
Starting item:  42
Ending item:  42
Starting item:  45
Ending item:  45
Starting item:  46
Ending item:  46
Starting item:  47
Ending item:  47
Starting item:  

KeyboardInterrupt: 